In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.dummy import DummyRegressor

# 1) Carregamento
df = pd.read_csv("energydata_complete.csv")

# 2) Features / Target
y = df["Appliances"].copy()

# Remover colunas não numéricas e a data (que não é um preditor)
X = df.drop(columns=["Appliances"], errors="ignore")
if "date" in X.columns:
    X = X.drop(columns=["date"], errors="ignore")

# 3) Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# 4) Modelos
modelos = {
    "Regressão Linear": LinearRegression(),  # regressão
    "Árvore de Regressão": DecisionTreeRegressor(  # decision tree
        random_state=42
    ),
    "Random Forest": RandomForestRegressor(  # random forest
        n_estimators=150,
        random_state=42
    )
}

# 5) Treino e avaliação
def avalia(model, Xtr, Xte, ytr, yte):
    model.fit(Xtr, ytr)
    pred = model.predict(Xte)

    r2 = r2_score(yte, pred)
    rmse = float(np.sqrt(mean_squared_error(yte, pred)))
    mae = float(mean_absolute_error(yte, pred))

    return {"R2": r2, "RMSE": rmse, "MAE": mae}

resultados = {}
for nome, modelo in modelos.items():
    resultados[nome] = avalia(modelo, X_train, X_test, y_train, y_test)

# Ordena por RMSE (menor melhor)
ordenados = sorted(resultados.items(), key=lambda kv: kv[1]["RMSE"])

# 6) Exibição
print("=== Comparativo de Modelos (Teste) ===")
for nome, mets in ordenados:
    print(f"{nome:20s} | R² = {mets['R2']:.4f} | RMSE = {mets['RMSE']:.2f} | MAE = {mets['MAE']:.2f}")

=== Comparativo de Modelos (Teste) ===
Random Forest        | R² = 0.5353 | RMSE = 68.20 | MAE = 32.69
Árvore de Regressão  | R² = 0.1798 | RMSE = 90.60 | MAE = 38.87
Regressão Linear     | R² = 0.1693 | RMSE = 91.17 | MAE = 52.55


No nosso teste, o Random Forest se saiu muito melhor que os outros modelos. Ele conseguiu explicar mais da metade da variação no consumo de energia da casa e errou menos nas previsões. Isso acontece porque ele combina várias árvores e consegue captar relações mais complexas entre as variáveis.

A Árvore de Regressão, sozinha, teve um desempenho bem mais fraco: acertou pouco da variação real e ainda errou bastante. Já a Regressão Linear foi a pior de todas, mostrando que o consumo de energia não segue uma relação simples e direta com as variáveis do ambiente.

Em resumo: se a ideia é prever o uso de energia de forma confiável, o Random Forest é claramente a escolha mais adequada.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# 1) Carregar dados
df = pd.read_csv("smart_grid_stability_augmented.csv")

# 2) Features e alvo
y_text = df["stabf"].astype(str)
# Mapeia: 0=stable, 1=unstable (nos interessa detectar instabilidade)
y = y_text.map({"stable": 0, "unstable": 1}).astype(int)

X = df.drop(columns=["stabf"])
# Garantir que só fiquem numéricas (caso haja algo residual)
X = X.select_dtypes(include=[np.number]).copy()

# 3) Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# 4) Modelos (simples)
modelos = {
    "Árvore de Decisão": DecisionTreeClassifier(random_state=42),
    "KNN (k=7)": Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=7))
    ]),
    "Regressão Logística": Pipeline([
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(max_iter=1000, random_state=42))
    ]),
}

# 5) Treinar e avaliar
def avaliar(modelo, Xtr, Xte, ytr, yte):
    modelo.fit(Xtr, ytr)
    pred = modelo.predict(Xte)

    acc = accuracy_score(yte, pred)
    f1u = f1_score(yte, pred, pos_label=1)  # F1 para "instável"
    cm = confusion_matrix(yte, pred, labels=[0,1])  # [[TN, FP],[FN, TP]]

    return acc, f1u, cm

resultados = {}
for nome, mdl in modelos.items():
    acc, f1u, cm = avaliar(mdl, X_train, X_test, y_train, y_test)
    resultados[nome] = {"Acurácia": acc, "F1_instável": f1u, "MatrizConfusão": cm}

# 6) Exibir resultados (limpo)
print("=== Classificação: Estabilidade da Rede (0=stable, 1=unstable) ===")
for nome, mets in resultados.items():
    cm = mets["MatrizConfusão"]
    print(f"\n{nome}")
    print(f"Acurácia:     {mets['Acurácia']:.4f}")
    print(f"F1 (instável): {mets['F1_instável']:.4f}")
    print("Matriz de Confusão [linhas=Real, colunas=Pred]:")
    print(pd.DataFrame(
        cm,
        index=["Real:0(stable)", "Real:1(unstable)"],
        columns=["Pred:0(stable)", "Pred:1(unstable)"]
    ))


=== Classificação: Estabilidade da Rede (0=stable, 1=unstable) ===

Árvore de Decisão
Acurácia:     1.0000
F1 (instável): 1.0000
Matriz de Confusão [linhas=Real, colunas=Pred]:
                  Pred:0(stable)  Pred:1(unstable)
Real:0(stable)              4344                 0
Real:1(unstable)               0              7656

KNN (k=7)
Acurácia:     0.9557
F1 (instável): 0.9657
Matriz de Confusão [linhas=Real, colunas=Pred]:
                  Pred:0(stable)  Pred:1(unstable)
Real:0(stable)              4004               340
Real:1(unstable)             191              7465

Regressão Logística
Acurácia:     0.9993
F1 (instável): 0.9995
Matriz de Confusão [linhas=Real, colunas=Pred]:
                  Pred:0(stable)  Pred:1(unstable)
Real:0(stable)              4338                 6
Real:1(unstable)               2              7654


No teste que fizemos, a Árvore de Decisão acertou absolutamente tudo, mas esse tipo de modelo costuma “decorar” demais os dados e pode não se sair tão bem em situações novas. O KNN teve um bom desempenho, mas cometeu mais erros na hora de identificar instabilidades. Já a Regressão Logística ficou praticamente perfeita: simples, clara e quase sem falhas, o que mostra que ela é a opção mais confiável para detectar quando a rede está instável.

Ou seja: se fosse para usar em uma aplicação real, o modelo mais seguro seria a Regressão Logística, porque une precisão com confiança para generalizar melhor.